In [ ]:
import os,sys
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

proj_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if proj_root not in sys.path:
    sys.path.insert(0, proj_root)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import logistic
from scipy.special import logit


from sklearn.datasets import make_blobs

import torch
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset
from torch.cuda.amp import autocast, GradScaler

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Train with GPU support.")
else:
    device = torch.device('cpu')
    print("No GPU found, train with CPU support.")

import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt


# own utils
from utils.graph import *
from utils.tram_models import *
from utils.tram_model_helpers import *
from utils.tram_data import *
from utils.continous import *
from utils.sampling_tram_data import *

Train with GPU support.


In [ ]:
def dgp_sklearn(nobs=1000, nvars=7, seed=42):
    X, _ = make_blobs(n_samples=nobs, n_features=nvars, centers=1,cluster_std=1.0, random_state=seed)
    cols = [f'x{i+1}' for i in range(nvars)]
    return pd.DataFrame(X, columns=cols)

df=dgp_sklearn(nobs=1000, nvars=2, seed=42)
print(df.info())


data_type= {'x1':'cont','x2':'cont'} # cont:continous, ord:ordinal, oher:everything else than images


train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

quantiles = train_df.quantile([0.05, 0.95])
min_vals = quantiles.loc[0.05]
max_vals = quantiles.loc[0.95]

In [ ]:
def load_and_write_test_dict_to_json(input:dict, test_name,file_path = 'twovars_model_loader_test_dict.json'):
    """
    input has to bea dictionary with the following structure:
                                "input":{
                                                'x1': {
                                                'data_type': 'cont',
                                                'node_type': 'source',
                                                'parents': [],
                                                'parents_datatype': {},
                                                'transformation_terms_in_h()': {},
                                                'transformation_term_nn_models_in_h()': {}},
                                                
                                For n nodes
    """
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
            
    except Exception as e:
        print(f"Error loading JSON file: {e}")
    
    data[test_name]={}
    data[test_name].setdefault("input", {})
    data[test_name].setdefault("output", {})

    
    data[test_name]["input"] = input
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for node_name in input:
        model = get_fully_specified_tram_model(node_name, input, verbose=False).to(device)
        data[test_name]["output"][node_name] = repr(model)
    
    try:
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=4)
        print(f"Updated outputs written to {file_path}")
    except Exception as e:
        print(f"Error writing to JSON file: {e}")
        
        
def run_model_loader_test(test_name: str, testdict_path: str, device: torch.device = None):
    """
    General test for fully_specified_tram_model loader based on ground-truth JSON.

    Args:
        test_name: Key in the JSON file identifying the test case.
        testdict_path: Path to the JSON file containing input and expected output.
        device: Torch device to move models to; defaults to CUDA if available, else CPU.

    Raises:
        AssertionError: If any model's repr() does not match the expected output.
        ValueError: If the test_name is not found in the JSON file.
    """
    # Load ground-truth data
    with open(testdict_path, 'r') as f:
        test_data = json.load(f)

    if test_name not in test_data:
        raise ValueError(f"Test name '{test_name}' not found in {testdict_path}")
    

    inputs = test_data[test_name]['input']
    expected_outputs = test_data[test_name].get('output', {})

    # Determine device
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Iterate and compare
    for node_name in inputs:
        model = get_fully_specified_tram_model(node_name, inputs, verbose=False).to(device)
        actual_repr = repr(model)
        expected_repr = expected_outputs.get(node_name)

        assert expected_repr is not None, (
            f"No expected output found for node '{node_name}' "
            f"in '{testdict_path}'."
        )
        assert actual_repr == expected_repr, (
            f"Mismatch for node '{node_name}':\n"
            f"  Expected: {expected_repr}\n"
            f"  Actual:   {actual_repr}"
        )
        


# Testing 2 variables
4 tests SI LS, CS and CI

In [ ]:
# LS
input={
        'x1': {
        'data_type': 'cont',
        'node_type': 'source',
        'parents': [],
        'parents_datatype': {},
        'transformation_terms_in_h()': {},
        'transformation_term_nn_models_in_h()': {}},
        'x2': {
        'data_type': 'cont',
        'node_type': 'sink',
        'parents': ['x1'],
        'parents_datatype': {'x1': 'cont'},
        'transformation_terms_in_h()': {'x1': 'ls'},
        'transformation_term_nn_models_in_h()': {'x1': 'LinearShift'}}
        }

load_and_write_test_dict_to_json(input, "test_2",file_path = 'twovars_model_loader_test_dict.json')

In [ ]:
# CS
input={
        'x1': {
        'data_type': 'cont',
        'node_type': 'source',
        'parents': [],
        'parents_datatype': {},
        'transformation_terms_in_h()': {},
        'transformation_term_nn_models_in_h()': {}},
        'x2': {
        'data_type': 'cont',
        'node_type': 'sink',
        'parents': ['x1'],
        'parents_datatype': {'x1': 'cont'},
        'transformation_terms_in_h()': {'x1': 'ls'},
        'transformation_term_nn_models_in_h()': {'x1': 'ComplexShiftDefaultTabular'}}
        }

load_and_write_test_dict_to_json(input, "test_2",file_path = 'twovars_model_loader_test_dict.json')

In [ ]:
# CI
input={
        'x1': {
        'data_type': 'cont',
        'node_type': 'source',
        'parents': [],
        'parents_datatype': {},
        'transformation_terms_in_h()': {},
        'transformation_term_nn_models_in_h()': {}},
        'x2': {
        'data_type': 'cont',
        'node_type': 'sink',
        'parents': ['x1'],
        'parents_datatype': {'x1': 'cont'},
        'transformation_terms_in_h()': {'x1': 'ci'},
        'transformation_term_nn_models_in_h()': {'x1': 'ComplexInterceptDefaultTabular'}}
        }

load_and_write_test_dict_to_json(input, "test_3",file_path = 'twovars_model_loader_test_dict.json')

In [44]:
# this tests all the 3 cases for SI LS CS and CI

for test in ['test_1', 'test_2', 'test_3']:
    run_model_loader_test(test, 'twovars_model_loader_test_dict.json')


# 3vars

In [ ]:
def get_base_model_class(class_name: str):
    # Strip digits to get the base class name
    for i, c in enumerate(class_name):
        if c.isdigit():
            return class_name[:i]
    return class_name

# --------- Group features by h_term base ---------
def group_by_base(term_dict, prefixes):
    if isinstance(prefixes, str):
        prefixes = (prefixes,)
    groups = defaultdict(list)
    for feat, conf in term_dict.items():
        h_term = conf['h_term']
        for prefix in prefixes:
            if h_term.startswith(prefix):
                if len(h_term) > len(prefix) and h_term[len(prefix)].isdigit():
                    key = h_term[:len(prefix)+1]
                else:
                    key = h_term
                groups[key].append((feat, conf))
                break
    return groups


def get_fully_specified_tram_model(node:str,target_nodes:dict,verbose=True): 

    ### iF node is a source -> Modeling as SimpleIntercept
    if target_nodes[node]['node_type'] == 'source':
        nn_int = SimpleIntercept()
        tram_model = TramModel(nn_int, None)  
        if verbose:
            print('>>>>>>>>>>>>  source node --> only  modelled only  by si') if verbose else None
            print(tram_model)
        return tram_model
    
    else:
        # read terms and model names form the config
        _,terms_dict,model_names_dict=ordered_parents(node, target_nodes)
        print(terms_dict)
        print(model_names_dict)
        
        # Combine terms and model names and divide in intercept and shift terms
        model_dict=merge_transformation_dicts(terms_dict, model_names_dict)
        
        
        
        
        # separate intercept and shift terms
        intercepts_dict = {k: v for k, v in model_dict.items() if "ci" in v['h_term'] or 'si' in v['h_term']}        
        shifts_dict = {k: v for k, v in model_dict.items() if "ci" not in v['h_term'] and  'si' not in v['h_term']}        
        
        
        print(intercepts_dict)
        print(shifts_dict)
        
        # make sure that nns are correctly defined afterwards
        nn_int, nn_shifts_list = None, []
        
        
        # --------- INTERCEPT TERM ---------
        intercept_groups = group_by_base(intercepts_dict, 'ci')

        if not intercept_groups:
            print('>>>>>>>>>>>> No ci detected --> intercept defaults to si') if verbose else None
            nn_int = SimpleIntercept()
        else:
            if len(intercept_groups) > 1:
                raise ValueError("Multiple intercept models detected; only one is supported.")


            group = list(intercept_groups.values())[0]
            
            print(f'group; {group}')
            
            any_class_name = group[0][1]['class_name']
            base_class_name = get_base_model_class(any_class_name)

            model_cls = globals()[base_class_name]
            n_features = len(group)
            nn_int = model_cls(n_features=n_features)

        # --------- SHIFT TERMS ---------
        shift_groups = group_by_base(shifts_dict, prefixes=('cs', 'ls'))
        
        print(shift_groups)

        for group in shift_groups.values():
            any_class_name = group[0][1]['class_name']
            base_class_name = get_base_model_class(any_class_name)

            model_cls = globals()[base_class_name]
            n_features = len(group)
            model = model_cls(n_features=n_features)
            nn_shifts_list.append(model)

        # --------- COMBINE TO NN CLASS ---------
        tram_model = TramModel(nn_int, nn_shifts_list)
        print('>>> TRAM MODEL:\n',tram_model) if verbose else None
        return tram_model

In [58]:
# TODO here is a bug when 2 linera shifts are used in the same model, e.g. when we have a model like this:

#  X3 ~ LS(X1) + LS(X2)
input={
        'x1': {
        'data_type': 'cont',
        'node_type': 'source',
        'parents': [],
        'parents_datatype': {},
        'transformation_terms_in_h()': {},
        'transformation_term_nn_models_in_h()': {}},
        'x2': {
        'data_type': 'cont',
        'node_type': 'source',
        'parents': [],
        'parents_datatype': {},
        'transformation_terms_in_h()': {},
        'transformation_term_nn_models_in_h()': {}},
        'x3': {
        'data_type': 'cont',
        'node_type': 'sink',
        'parents': ['x1','x2'],
        'parents_datatype': {'x1': 'cont','x2': 'cont'},
        'transformation_terms_in_h()': {'x1': 'ls','x2': 'ls'},
        'transformation_term_nn_models_in_h()': {'x1': 'LinearShift','x2': 'LinearShift'}}
        }

load_and_write_test_dict_to_json(input, "test_1", file_path = 'threevars_model_loader_test_dict.json')

OrderedDict([('x1', 'ls'), ('x2', 'ls')])
OrderedDict([('x1', 'LinearShift'), ('x2', 'LinearShift')])
{}
{'x1': {'h_term': 'ls', 'class_name': 'LinearShift'}, 'x2': {'h_term': 'ls', 'class_name': 'LinearShift'}}
defaultdict(<class 'list'>, {'ls': [('x1', {'h_term': 'ls', 'class_name': 'LinearShift'}), ('x2', {'h_term': 'ls', 'class_name': 'LinearShift'})]})
Updated outputs written to threevars_model_loader_test_dict.json


In [ ]:
input=






In [ ]:
data_type = {'x2': 'cont','x3': 'cont','x4': 'cont','x5': 'cont','x6': 'cont','x7': 'cont','x8': 'cont','x9': 'cont'}
adj_matrix = np.array([
    ["0",  "0",  "0",  "0", "0",  "0",  "0",  "ls"],   # x1 → x2 (ci), x1 → x3 (ls)
    ["0",  "0",  "0",  "0", "0",  "0",  "0",  "cs12"],
    ["0",  "0",  "0",  "0", "0",  "0",  "0",  "cs21"],
    ["0",  "0",  "0",  "0", "0",  "0",  "0",  "cs22"],
    ["0",  "0",  "0",  "0", "0",  "0",  "0",  "ci12"],
    ["0",  "0",  "0",  "0", "0",  "0",  "0",  "ci11"],
    ["0",  "0",  "0",  "0", "0",  "0",  "0",  "cs11"],
    ["0",  "0",  "0",  "0", "0",  "0",  "0",  "0"]
], dtype=object)

plot_seed = 42
plot_dag(adj_matrix, data_type, seed=plot_seed)

In [ ]:
# check if there are Ci or Compelx shifts in the models. If yes define the modelnames
nn_names_matrix= create_nn_model_names(adj_matrix,data_type)
plot_nn_names_matrix(nn_names_matrix,data_type)

In [ ]:
for node in conf_dict:

tram_model = get_fully_specified_tram_model(node, conf_dict, verbose=True).to(device)
